In [ ]:
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import warnings
import seaborn as sns
import matplotlib.pylab as plt
import PIL
import zipfile
import glob
from sklearn.model_selection import StratifiedKFold, KFold 
from sklearn.metrics import f1_score
from keras import backend as K
from keras import layers, models, optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import *
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

warnings.filterwarnings('ignore')
K.image_data_format()

In [ ]:
# distribution 확인
# y Label
DATA_PATH      = "../input/"
TRAIN_IMG_PATH = os.path.join(DATA_PATH, 'train')
TEST_IMG_PATH  = os.path.join(DATA_PATH, 'test')

In [ ]:
# CSV 파일 경로를 통해 data read : train, test, class
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
df_class = pd.read_csv(os.path.join(DATA_PATH, 'class.csv'))

In [ ]:
# train.csv, test.csv : bounding Box axis, img_file 존재.
df_train.head()

In [ ]:
# class.csv : 차종 정보 및 class index. 
df_class.head()

In [ ]:
print("train dataset : ",  df_train.shape)
print("test dataset : ",  df_test.shape)

In [ ]:
# 파이썬에서 이미지 보고 싶은 경우,
# PIL library ->  이미지 로드하는 라이브러리라고 생각하자
# PIL library가 좀 더 성능면에서 좋음.

# 전반적으로 차에 대한 이미지는 잘 보임. 
# 	-> bounding box는 필요성 보임 
# 	-> why ? 116번 이미지 같은 경우, 주위에 차들이 또 보임..

import PIL
from PIL import ImageDraw

tmp_imgs = df_train['img_file'][200:210] # 이미지 10개 사용
plt.figure(figsize = (12, 20))           # 사진 size 설정
for num, f_name in enumerate(tmp_imgs):  
    img = PIL.Image.open(os.path.join(TRAIN_IMG_PATH, f_name)) # PIL library 의 image open 함수 이용 PATH, NAME 필요
    plt.subplot(5, 2, num + 1) # 5행 1열중, num + 1 째 위치(?)
    plt.title(f_name)          # Image title 설정
    plt.imshow(img)            # show
    plt.axis('off')            # 축은 안보이게 off
    
# 차의 정면, 후면, 옆면 등 다양하게 있음. 
# bounding Box 처리를 통해 cropping의 필요성 존재.

In [ ]:
# 사람들은 보통 어떤 것들을 보고 차종을 분류할까?
#    -> 차의 크기
#    -> 마크** : 각 브랜드마다 차의 특성들이 조금씩 있는데, 주로 마크가 있음. 
#    -> 번호판
#    -> 문짝 개수도 힌트가 될 수 있음.
#    -> 휠

# 참고 URL
# https://www.motorgraph.com/news/articleView.html?idxno=5215 : 차종 분류 어떻게 구분하나..?
# -> 차의 box 개념이 존재해서 이를 통해 구분 가능. 

In [ ]:
# y 레이블 분포도 확인
# -> 전반적으로 고른 분포 형태를 가지고 있음.
train_class = df_train["class"]
plt.figure(figsize = (12, 6))
sns.countplot(train_class, 
              order = df_train["class"].value_counts(ascending = True).index
             )
plt.title("Number of data per each class")
plt.show()

In [ ]:
# max : 83개.
# min : 30개.
# mean : 50개. -> 9990 // 196 : 50 개 정도 되므로, 굉장히 이상적인 이미지 Label 분포

yLabel_count = df_train["class"].value_counts(ascending = True)
print("yLabel count max : " , yLabel_count.max())
print("yLabel count min : " , yLabel_count.min())
print("yLabel count mean : ", yLabel_count.mean())

In [ ]:
# data check
# 실제 csv 파일에 정리된 img_file과, 실제 폴더 내에 .img 로 형성되어 있는지 확인 필요.
if set(list(df_train.img_file)) == set(os.listdir(TRAIN_IMG_PATH)) :
    print("Train file 누락 없음!")
else:
    print("Train file 누락!")

if set(list(df_test.img_file)) == set(os.listdir(TEST_IMG_PATH)) :
    print("Test file 누락 없음!")
else:
    print("Test file 누락!")

In [ ]:
# Bounding Box and Cropping
# 허태명님, 태진님 커널 참조.

# Bounding Box
# 이미지 내부에서 특정 object를 박스로 레이블한 좌표
# 좌측 상단, 우측 하단 좌표가 주어진다
# 좌표는 이미지의 픽셀 좌표 

# draw_rect
# 이미지의 Bounding Box 를 그려주는 함수 
# @param drawcontext -> 그리고자 하는 img 객체(ImageDraw.Draw(img))
# @param pos,        -> [x1, y1, x2, y2]
# @param outline     -> 선의 색깔
# @param width       ->  선의 폭
def draw_rect(drawcontext, pos, outline = None, width = 0) : #
    (x1, y1) = (pos[0], pos[1]) # x1, y1 입력
    (x2, y2) = (pos[2], pos[3]) # x2, y2 입력
    points   =  (x1, y1), (x2, y1), (x2, y2), (x1, y2), (x1, y1) # 선을 4개 긋기 위한 좌표 5개. 다시 (x1, y1) 으로 회귀
    drawcontext.line(points, fill = outline, width = width) # drawcontext.line function

In [ ]:
# make_boxing_img function
# bounding box를 적용한 이미지 출력 함수.
# 기존 좌표로 주어진, (x1, y1, x2, y2) 의 bounding axis가 제대로 되어 있는지? 함 확인하고자 만든 함수.
# @param img_name -> 이미지 이름

def make_boxing_img(img_name):
    # img가 train, test 인지 여부에 따라 PATH, data 각각 저장
    if img_name.split('_')[0] == "train" :
          PATH = TRAIN_IMG_PATH
          data = df_train
    elif img_name.split('_')[0] == "test" :
        
        PATH = TEST_IMG_PATH
        data = df_test
    
    # 선이 그려지지 않은 이미지 한개를 open 할 객체를 img에 저장
    img = PIL.Image.open(os.path.join(PATH, img_name))
    # reshape 함수는 Python을 통해 머신러닝 혹은 딥러닝 코딩을 하다보면 꼭 나오는 numpy 내장 함수입니다.
    #  reshape(-1) 인 경우 -> 구조화 되어 있는 dataframe을 1차원 배열을 반환  
    pos = data.loc[data["img_file"] == img_name, ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2']].values.reshape(-1) # values : header를 제거하고 값만 가져오는 함수.
    draw = ImageDraw.Draw(img)                        # 이미지를 그리고,
    draw_rect(draw, pos, outline = 'red', width = 10) # Bounding Box를 생성
        
    return img

In [ ]:
f_name = "train_00102.jpg" # 이미지 샘플 1개 이름 저장
plt.figure(figsize = (20, 10)) # 이미지 크기 지정 
plt.subplot(1, 2, 1)

# Original Image
origin_img = PIL.Image.open(os.path.join(TRAIN_IMG_PATH, f_name))
plt.title("Original Image - {}".format(f_name))
plt.imshow(origin_img)
plt.axis('off')

# Image included bounding box
plt.subplot(1, 2, 2)
boxing = make_boxing_img(f_name)
plt.title("Boxing Image - {}".format(f_name))
plt.imshow(boxing)
plt.axis('off')

plt.show()

In [ ]:
##  실제 cropping을 통해, output data를 생성하기
# idea. crop function 사용

def crop_boxing_img(img_name, margin = 16):
    if img_name.split('_')[0] == "train" :
        PATH = TRAIN_IMG_PATH
        data = df_train
    elif img_name.split('_')[0] == "test" :
        PATH = TEST_IMG_PATH
        data = df_test
        
    img = PIL.Image.open(os.path.join(PATH, img_name))
    pos = data.loc[data["img_file"] == img_name, 
                   ['bbox_x1','bbox_y1', 'bbox_x2', 'bbox_y2']].values.reshape(-1) # img_name가 matching된 좌표 값을 1차원 vector로 변경
    
    
    # margin 값을 기준으로, 사진이 벗어나면 안되기 때문에 이에 대한 계산 값 필요.
    # (x1, y1), (x2, y2)
    width, height = img.size     # 기존 사진의 width, height 값.
    x1 = max(0, pos[0] - margin)
    y1 = max(0, pos[1] - margin)
    x2 = min(pos[2] + margin, width)
    y2 = min(pos[3] + margin, height)
    
    # 기존 csv에 작성 되어있는 좌표 값이 이상할 수도 있기 때문에 이것도 check
    # 1. x2 - x1 이 width 보다 작은지? 
    # 2. y2 - y1 이 height 보다 작은지?
    # 만약 cropping 할 수 없으면, 기존 img를 return
    if abs(pos[2] - pos[0]) > width or  abs(pos[3] - pos[1]) > height:
        print(f'{img_name} is wrong bounding box, img size: {img.size},  bbox_x1: {pos[0]}, bbox_x2: {pos[2]}, bbox_y1: {pos[1]}, bbox_y2: {pos[3]}')
        return img
    
    return img.crop((x1,y1,x2,y2))

In [ ]:
# train img data cropping
# PIL module 중, 하나인 img.crop, img.crop.save 가 가능한 듯
# crop.save function : cropping img 저장.

for i, row in df_train.iterrows():
    cropped = crop_boxing_img(row['img_file'])
    cropped.save(row['img_file'])

In [ ]:
# test img data cropping
for i, row in df_test.iterrows():
    cropped = crop_boxing_img(row['img_file'])
    cropped.save(row['img_file'])

In [ ]:
# 실제 cropping 된 이미지 확인
# train_data cropping check
tmp_imgs = df_train['img_file'][100:105]
plt.figure(figsize=(12,20))

for num, f_name in enumerate(tmp_imgs):
    img = PIL.Image.open(os.path.join(TRAIN_IMG_PATH, f_name))
    plt.subplot(5, 2, 2*num + 1)
    plt.title(f_name)
    plt.imshow(img)
    plt.axis('off')
    
    img_crop = PIL.Image.open(f_name)
    plt.subplot(5, 2, 2*num + 2)
    plt.title(f_name + ' cropped')
    plt.imshow(img_crop)
    plt.axis('off')

In [ ]:

# test_data check
tmp_imgs = df_test['img_file'][115:120]
plt.figure(figsize=(12,20))

for num, f_name in enumerate(tmp_imgs):
    img = PIL.Image.open(os.path.join(TEST_IMG_PATH, f_name))
    plt.subplot(5, 2, 2*num + 1)
    plt.title(f_name)
    plt.imshow(img)
    plt.axis('off')
    
    img_crop = PIL.Image.open(f_name)
    plt.subplot(5, 2, 2*num + 2)
    plt.title(f_name + ' cropped')
    plt.imshow(img_crop)
    plt.axis('off')

In [ ]:
# cropping data zip file 
with zipfile.ZipFile('train_crop.zip','w') as zip: 
        # writing each file one by one 
        for file in glob.glob('train*.jpg'): 
            zip.write(file)
            
with zipfile.ZipFile('test_crop.zip','w') as zip: 
        # writing each file one by one 
        for file in glob.glob('test*.jpg'): 
            zip.write(file)

In [ ]:
!rm -rf *.jpg